In [1]:
import random, glob
import numpy as np

In [2]:
music_files = [a for a in glob.glob("dataset/*/*")]
print("A random song", random.sample(music_files, 1))

A random song ['dataset\\tschai\\ty_januar.mid']


In [3]:
from music21 import midi
def play_midi_file(midi_file_name):
    mf = midi.MidiFile()

    mf.open(midi_file_name) # path='abc.midi'
    mf.read()
    mf.close()
    s = midi.translate.midiFileToStream(mf)
    s.show('midi')

In [4]:
len(music_files)

300

In [5]:
from music21 import converter,corpus, chord

In [6]:
def get_score(music_files):
    chords = []
    for file_no, filename in enumerate(music_files):
        try:
            chords.append(converter.parse(filename))
            print(f'Happening of {file_no}', "filename = ", filename)
            
        except:
            print(f'Happening of {file_no}', "filename = ", filename)
            print("file failed!!!!!")
            continue
        
    
    return chords

In [7]:
music_files_5 = music_files[:5]
#get_chords(music_files[:10])

In [8]:
score_of_all_musics = get_score(music_files_5)

Happening of 0 filename =  dataset\albeniz\alb_esp1.mid
Happening of 1 filename =  dataset\albeniz\alb_esp1.song
file failed!!!!!
Happening of 2 filename =  dataset\albeniz\alb_esp2.mid
Happening of 3 filename =  dataset\albeniz\alb_esp3.mid
Happening of 4 filename =  dataset\albeniz\alb_esp4.mid


In [9]:
score_of_all_musics

[<music21.stream.Score 0x22d56f444c0>,
 <music21.stream.Score 0x22d586e0f40>,
 <music21.stream.Score 0x22d56f44d60>,
 <music21.stream.Score 0x22d56f442b0>]

In [10]:
from music21 import chord, duration

Classifying the music into different modes
- firstly, let's code for major mode

In [11]:
from music21 import *

In [31]:
def round_chord_durations(number):
    if(number>1.25):
        return 2
    if(number>.30):
        return 1.25
    if(number>.10):
        return 0.3
    return 0.1

For chord and duration of a single song(music file)

In [45]:
def get_chord_and_duration_data(individual_score):
    chord_duration_data = []
    note_and_chord_sequence =[]
    
    note_and_chord_duration =[]   ## not for now:
    all_tempo= []
    
    #to see the number of tracks:
    print(len(individual_score.parts))
    
    for element in individual_score.flat:
        #print("element = ", element, type(element))
        
        if isinstance(element, chord.Chord):
            note_and_chord_sequence.append('<SOC>') ## Start of Chord
            [note_and_chord_sequence.append(pitch.nameWithOctave) for pitch in element.pitches]
            note_and_chord_sequence.append('<EOC>') ## End of Chord
            
            chord_duration =round_chord_durations(element.duration.quarterLength)
            note_and_chord_sequence.append(chord_duration)
            #print(chord_name, chord_duration)
            
        elif isinstance(element, note.Note):
            [note_and_chord_sequence.append(pitch.nameWithOctave) for pitch in element.pitches]
    
            note_duration = round_chord_durations(element.duration.quarterLength)
            note_and_chord_sequence.append(note_duration)
            #print(note_name, note_duration)
        
        elif isinstance(element, note.Rest):
            rest_note_name = element.name
            #print(rest_note_name)
            
        elif isinstance(element, tempo.MetronomeMark):
            tempo_bpm = element.getQuarterBPM()
            all_tempo.append(tempo_bpm)
            #print(tempo_bpm)
        else:
            print(element, type(element))
            
    
    print("tempo ko lagi = " ,np.quantile(all_tempo, .25), np.quantile(all_tempo, .50), np.quantile(all_tempo, .75) )
    print(note_and_chord_sequence)
    return note_and_chord_sequence

In [47]:
#get_chord_and_duration_data(score_of_all_musics[0])

In [48]:
chords_and_duration_data_all_music= []
for one_score in score_of_all_musics:
   chords_and_duration_data_all_music.append((get_chord_and_duration_data(one_score)))

2
<music21.metadata.Metadata object at 0x22d56f448e0> <class 'music21.metadata.Metadata'>
Piano right: Piano right <class 'music21.instrument.Piano'>
Piano <class 'music21.instrument.Piano'>
Piano left: Piano left <class 'music21.instrument.Piano'>
Piano <class 'music21.instrument.Piano'>
<music21.clef.TrebleClef> <class 'music21.clef.TrebleClef'>
<music21.clef.TrebleClef> <class 'music21.clef.TrebleClef'>
F major <class 'music21.key.Key'>
F major <class 'music21.key.Key'>
<music21.meter.TimeSignature 3/4> <class 'music21.meter.base.TimeSignature'>
<music21.meter.TimeSignature 3/4> <class 'music21.meter.base.TimeSignature'>
<music21.bar.Barline type=final> <class 'music21.bar.Barline'>
<music21.bar.Barline type=final> <class 'music21.bar.Barline'>
tempo ko lagi =  106.5 118.0 132.125
['A5', 1.25, 'A3', 1.25, 'E6', 2, 'E4', 2, 'E6', 1.25, 'E4', 1.25, '<SOC>', 'D6', 'E6', '<EOC>', 0.3, '<SOC>', 'D4', 'E4', '<EOC>', 0.3, 'D6', 1.25, 'D4', 1.25, 'C6', 1.25, 'C4', 1.25, 'D6', 1.25, 'D4', 1.

In [69]:
# this is the main data:
chords_and_duration_data_all_music[0][1]
#np.asarray(chords_and_duration_data_all_music).shape

1.25

In [ ]:
print("Generating music from our processed chords...")
proccessed_chords_to_midi_sample = get_music_midi_filename_from_chords(chords_and_duration_data_all_music[0][4:50])
print(proccessed_chords_to_midi_sample)
play_midi_file(proccessed_chords_to_midi_sample)

from Tonic_mode_all, separating major and minor songs 

In [17]:
tonic_mode_all[0][0], tonic_mode_all[0][-1]

NameError: name 'tonic_mode_all' is not defined

In [18]:
all_major_songs_chords_and_duration = []
all_minor_songs_chords_and_duration = []
for i in range(len(tonic_mode_all)):
    if(tonic_mode_all[i][-1] == 'major'):
        try:
            all_major_songs_chords_and_duration.append(chords_and_duration_data_all_music[i])
        except:
            all_major_songs_chords_and_duration.append('NANNNNNN')
    else:
        try:
            all_minor_songs_chords_and_duration.append(chords_and_duration_data_all_music[i])
        except:
             all_minor_songs_chords_and_duration.append('NANNNNNN')

NameError: name 'tonic_mode_all' is not defined

In [19]:
len(all_minor_songs_chords_and_duration), #minor_songs_chords_with_duration, len(minor_songs_chords_with_duration)

(0,)

In [20]:
major_dataset = all_major_songs_chords_and_duration 
minor_dataset = all_minor_songs_chords_and_duration

In [21]:
#just a reference code
#major_dataset
#y=np.array(major_dataset)
#unique = set(major_dataset_all)
#unique2 = set(minor_dataset_all)

In [22]:
chords_and_duration_data_all_music

[[], [], [], []]

In [70]:
main_dataset = chords_and_duration_data_all_music

* Data preparation stage

In [71]:
no_of_timesteps = 32
x = []
y = []
## CD stands for chord and duration.

for each_music_with_CD in main_dataset:
    for each_CD in range(0, len(each_music_with_CD) - no_of_timesteps,  1):
        
        ## preparing input and output sequences:
        input_ = each_music_with_CD[i:i + no_of_timesteps]
        output = each_music_with_CD[i + no_of_timesteps]
        
        for i in range(len(input_)):
            input_[i]= "@".join(map(str,input_[i]))
            
        output = "@".join(map(str,output))
        
        x.append(input_)
        y.append(output)
        
x=np.array(x)
y=np.array(y)

NameError: name 'i' is not defined

In [25]:
y

array([], dtype=float64)

In [26]:
#assigning unique integer to every chords_and_duration

unique_x_CD = list(set(x.ravel()))
unique_x_CD_to_int = dict((chord_and_duration, number) for number, chord_and_duration in enumerate(unique_x_CD))
unique_x_CD_to_int

{}

In [27]:
#preparing input sequences::

x_seq=[]
for each_row in x:
    temp=[]
    for each_piece in each_row:
        #assigning unique integer to every note
        temp.append(unique_x_CD_to_int[each_piece])
    x_seq.append(temp)
    
x_seq = np.array(x_seq)
x_seq.shape

(0,)

In [28]:
# preparing th output sequences as well::

unique_y_CD = list(set(y))
unique_y_CD_to_int = dict((chord_and_duration, number) for number, chord_and_duration in enumerate(unique_y_CD)) 
unique_y_CD_to_int

{}

In [29]:
y_seq=np.array([unique_y_CD_to_int[i] for i in y])
y_seq.shape

(0,)

In [30]:
# preserving 80% of the data for training and the rest 20% for the evaluation:

from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_seq,y_seq,test_size=0.2,random_state=0)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
len(x_val)

# model building phase:

In [31]:
import tensorflow as tf

In [32]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [33]:
from keras.layers import *
from keras.models import *
from keras.callbacks import *
import keras.backend as K
from tensorflow import keras

K.clear_session()
model = Sequential()
    
#embedding layer
model.add(Embedding(len(unique_x_CD), 100, input_length=32,trainable=True)) 

model.add(Conv1D(64,3, padding='causal',activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))
    
model.add(Conv1D(128,3,activation='relu',dilation_rate=2,padding='causal'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))

model.add(Conv1D(256,3,activation='relu',dilation_rate=4,padding='causal'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))
          
#model.add(Conv1D(256,5,activation='relu'))    
model.add(GlobalMaxPool1D())
    
model.add(Dense(256, activation='relu'))
model.add(Dense(len(unique_y_CD), activation='softmax'))
    
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

model.summary()

ValueError: Both `input_dim` and `output_dim` should be positive, Received input_dim = 0 and output_dim = 100

In [35]:
def lstm():
    K.clear_session()
    model = Sequential()
    #embedding layer
    model.add(Embedding(len(unique_x_CD), 100, input_length=32,trainable=True)) 
    model.add(LSTM(128,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128))
    model.add(Dropout(0.2))
    model.add(Dense(256))
    model.add(Activation('relu'))

    model.add(Dense(len(unique_y_CD), activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
    return model

In [36]:
model = lstm()
model.summary()

ValueError: Both `input_dim` and `output_dim` should be positive, Received input_dim = 0 and output_dim = 100

In [37]:
keras.utils.plot_model(model, "my_first_model.png")

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [ ]:
!pip install pydot

In [ ]:
!pip install h5py

In [ ]:
import h5py

In [ ]:
#defining call back to save the best model during training>
mc=ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True,verbose=1)

In [ ]:
#actual training
history = model.fit(np.array(x_train),np.array(y_train),batch_size=128,epochs=10, 
                   validation_data=(np.array(x_val),np.array(y_val)),verbose=1, callbacks=[mc])

In [ ]:
#loading best model (Previously trained modle)
from keras.models import load_model
model = load_model('best_model.h5')


In [ ]:
import numpy as np
import random
ind = np.random.randint(0,len(x_val)-1)
random_music = x_val[ind]
random_music

In [ ]:
predictions=[]
for i in range(10):

    random_music = random_music.reshape(1,no_of_timesteps)
    print("random music = ", random_music)
    

    prob  = model.predict(random_music)[0]
    y_pred= np.argmax(prob,axis=0)
    predictions.append(y_pred)

    random_music = np.insert(random_music[0],len(random_music[0]),y_pred)
    random_music = random_music[1:]
    
print(predictions)

In [ ]:
#intergers back to notes
unique_x_int_to_CD = dict((number, note_) for number, note_ in enumerate(unique_x_CD)) 
unique_x_int_to_CD

In [ ]:
predicted_CD = [unique_x_int_to_CD[i] for i in predictions]
predicted_CD

In [ ]:
predicted_CD_split = []
for each_outcome in predicted_CD:
    temp_list = []
    temp_list = each_outcome.split("@")
    temp_list[1] =float(temp_list[1])
    predicted_CD_split.append(tuple(temp_list))
    

predicted_CD_split

In [ ]:
print("Generating music from our processed chords...")
proccessed_chords_to_midi_sample = get_music_midi_from_chords_and_duration(predicted_CD_split)
print(proccessed_chords_to_midi_sample)
play_midi_file(proccessed_chords_to_midi_sample)

In [ ]:
import random, glob
import numpy as np

In [ ]:
music_files = [a for a in glob.glob("dataset/*/*")]
print("A random song", random.sample(music_files, 1))

In [ ]:
len(music_files)

In [ ]:
from music21 import midi
def play_midi_file(midi_file_name):
    mf = midi.MidiFile()

    mf.open(midi_file_name) # path='abc.midi'
    mf.read()
    mf.close()
    s = midi.translate.midiFileToStream(mf)
    s.show('midi')

In [ ]:
# for seeing output from chords and given duration:
def get_music_midi_from_chords_and_duration(input_chords):
    midi_stream = stream.Stream()

    for note_pattern, duration_pattern in input_chords:
        notes_in_chord = note_pattern.split('.')
        
        chord_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(current_note)
            new_note.duration = duration.Duration(duration_pattern)
            new_note.storedInstrument = instrument.Violoncello()
            chord_notes.append(new_note)
        new_chord = chord.Chord(chord_notes)
        
        midi_stream.append(new_chord)

        new_tempo = tempo.MetronomeMark(number=50)
            
        midi_stream.append(new_tempo)

    midi_stream = midi_stream.chordify()
    timestr = time.strftime("%Y%m%d-%H%M%S")
    new_file = 'output-' + timestr + '.mid'
    return midi_stream.write('midi', fp=new_file)